# Text Generation with Neural Networks
In this notebook our objective is to demonstrate how to generate text using a character-based RNN working with a dataset of Shakespeare's  writing

# Notebook Overview
- Imports Dependencies
- Define Constants and Paths and Configure Logging
- Get Text Data
- Preparing textual data
- One Hot Encoding
- Creating Training Batches
- Creating the LSTM Model
- Training the Network
- Generating Predictions

## Imports Dependencies

In [ ]:
# Standard Library Imports
import logging

# Third-Party Libraries
import torch
from torch import nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt

# MLflow for Experiment Tracking and Model Management
import mlflow

torch.manual_seed(0)

## Define Constants and Paths and Configure Logging

In [ ]:
# Define global experiment and run names to be used throughout the notebook
EXPERIMENT_SET = "RNN text generation"
RUN_NAME = "RNN Text Generation"
MODEL_NAME = "dict_torch_rnn_model.pt"

# Set up the paths
DATA_PATH = "shakespeare.txt"
MODEL_DECODER_PATH = "models/decoder.pt"
MODEL_ENCODER_PATH = "models/encoder.pt"

# Set up the chunk separator for text processing
CHUNK_SEPARATOR = "\n\n"

In [ ]:
# Configure the logging module with desired format and level
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

# Create a logger for this notebook
logger = logging.getLogger('text-generation-Torch-notebook')
logger.info("Logging configured successfully")

## Get Text Data

This is the text we'll use as a basis for our generations: let's try to generate 'Shakespearean' texts.

This text is from Shakespeare's Sonnet 1. It's one of the 154 sonnets written by William Shakespeare that were first published in 1609. This particular sonnet, like many others, discusses themes of beauty, procreation, and the transient nature of life, urging the beautiful to reproduce so their beauty can live on through their offspring.

In [ ]:
with open(DATA_PATH,'r',encoding='utf8') as f:
    text = f.read()

In [ ]:
logger.info('First 600 chars: \n')
logger.info(text[:600])

## Preparing textual data

We need to encode our data to give the model a proper numerical representation of our text.

In [ ]:
all_characters = set(text) # creates a set of unique characters found in the text

In [ ]:
len(all_characters)

In [ ]:
decoder = dict(enumerate(all_characters))
# assigns a unique integer to each character in a dictionary format, 
# creating a mapping that can later be used to transform encoded predictions back into characters

In [ ]:
encoder = {char: ind for ind, char in decoder.items()} 
# reverses the decoder dictionary, providing a mapping from characters to their respective assigned integers, which is used to encode the text.

In [ ]:
torch.save(decoder, MODEL_DECODER_PATH)
torch.save(encoder, MODEL_ENCODER_PATH)

In [ ]:
encoded_text = np.array([encoder[char] for char in text])
# encodes the entire text as an array of integers, with each integer representing the character at that position
#in the text according to the encoder dictionary

## One Hot Encoding

One-hot encoding is a way to convert categorical data into a fixed-size vector of numerical values.

This encoding allows the model to treat input data uniformly and is particularly important for models that need to determine the presence or absence of a feature, such as a particular character.

In [ ]:
def one_hot_encoder(encoded_text, num_uni_chars):
    """
        Convert categorical data into a fixed-size vector of numerical values.

        Args:
            encoded_text: Batch of encoded text.
            num_uni_chars: Number of unique characters

    """
    try:
        # Create a placeholder for zeros
        one_hot = np.zeros((encoded_text.size, num_uni_chars))
        
        # Convert data type for later use with pytorch
        one_hot = one_hot.astype(np.float32)

        # Using indexing fill in the 1s at the correct index locations
        one_hot[np.arange(one_hot.shape[0]), encoded_text.flatten()] = 1.0
        
        # Reshape it so it matches the batch shape
        one_hot = one_hot.reshape((*encoded_text.shape, num_uni_chars))
        
        return one_hot
    except Exception as e:
            logger.error(f"Error converting categorical data: {str(e)}")


# Creating Training Batches

Training batches are a way of dividing the dataset into smaller, manageable groups of data points that are fed into a machine learning model during the training process.

In [ ]:
def generate_batches(encoded_text, samp_per_batch=10, seq_len=50):
    
    '''
    Generate (using yield) batches for training.
    
    X: Encoded Text of length seq_len
    Y: Encoded Text shifted by one
    
    Example:
    
    X:
    
    [[1 2 3]]
    
    Y:
    
    [[ 2 3 4]]
    
    encoded_text : Complete Encoded Text to make batches from
    batch_size : Number of samples per batch
    seq_len : Length of character sequence
       
    '''
    try:
        # Total number of characters per batch
        # Example: If samp_per_batch is 2 and seq_len is 50, then 100
        # characters come out per batch.
        char_per_batch = samp_per_batch * seq_len
        
        # Number of batches available to make
        # Use int() to roun to nearest integer
        num_batches_avail = int(len(encoded_text)/char_per_batch)
        
        # Cut off end of encoded_text that
        # won't fit evenly into a batch
        encoded_text = encoded_text[:num_batches_avail * char_per_batch]
        
        # Reshape text into rows the size of a batch
        encoded_text = encoded_text.reshape((samp_per_batch, -1))

        # Go through each row in array.
        for n in range(0, encoded_text.shape[1], seq_len):
            # Grab feature characters
            x = encoded_text[:, n:n+seq_len]
            # y is the target shifted over by 1
            y = np.zeros_like(x)
            try:
                y[:, :-1] = x[:, 1:]
                y[:, -1]  = encoded_text[:, n+seq_len]
            except:
                y[:, :-1] = x[:, 1:]
                y[:, -1] = encoded_text[:, 0]
                
            yield x, y
        logger.info("Batches generated successfully")
    except Exception as e:
            logger.error(f"Error Generating batches: {str(e)}")


# Creating the LSTM Model

In [ ]:
class CharModel(nn.Module):
    def __init__(self, all_chars, num_hidden=256, num_layers=4,drop_prob=0.5, use_gpu=False):
        """Initializes CharModel

        Args:
            all_chars: Set of unique characters found in the text.
            num_hidden: Number of hidden layers. Defaults to 256.
            num_layers: Number of layers. Defaults to 4.
            drop_prob: Regularization technique to prevent overfitting. Defaults to 0.5.
            use_gpu: If the model uses GPU. Defaults to False.
        """
        try:
            super().__init__()
            self.drop_prob = drop_prob
            self.num_layers = num_layers
            self.num_hidden = num_hidden
            self.use_gpu = use_gpu
            
            self.all_chars = all_chars
            self.decoder = torch.load(MODEL_DECODER_PATH)
            self.encoder = torch.load(MODEL_ENCODER_PATH)
            
            self.lstm = nn.LSTM(len(self.all_chars), num_hidden, num_layers, dropout=drop_prob, batch_first=True)
            self.dropout = nn.Dropout(drop_prob)
            self.fc_linear = nn.Linear(num_hidden, len(self.all_chars))
            logger.info("CharModel Initialized successfully")
    
        except Exception as e:
            logger.error(f"Error Initializing CharModel: {str(e)}")
      
    
    def forward(self, x, hidden):
        """Implementation of the CharModel logic, in which, the input passes through every step of the arquiteture

        Args:
            x: Input tensor with shape (batch size and senquency length) containing character indices.
            hidden: Tuple containing the inicial hidden states of the CharModel each with shape (batch size and senquency length).

        Returns:
            final_out: Output tensor representing the predicted logits for each character in the sequence.
            hidden: Tuple containing the final hidden states of the CharModel.
        """
        try:
            lstm_output, hidden = self.lstm(x, hidden)       
            drop_output = self.dropout(lstm_output)
            drop_output = drop_output.contiguous().view(-1, self.num_hidden)
            final_out = self.fc_linear(drop_output)
            
            return final_out, hidden
        
        except Exception as e:
            logger.error(f"Error implementing CharModel logic: {str(e)}")
    
    
    def hidden_state(self, batch_size):
        """
        Initializes and returns the initial hidden state for a recurrent neural network (e.g., LSTM).

        This method creates zero-filled tensors for the hidden state (h_0) and cell state (c_0), 
        supporting GPU execution if `self.use_gpu` is set to True.

        Args:
            batch_size: The number of sequences in the input batch, used to determine the tensor dimensions.

        Returns:
            Tuple: A tuple containing the hidden state and cell state tensors 
            with shape (num_layers, batch_size, num_hidden). Returns None if an exception occurs, and logs the error.
        """
        try:
            if self.use_gpu:
                hidden = (torch.zeros(self.num_layers,batch_size,self.num_hidden).cuda(),
                        torch.zeros(self.num_layers,batch_size,self.num_hidden).cuda())
            else:
                hidden = (torch.zeros(self.num_layers,batch_size,self.num_hidden),
                        torch.zeros(self.num_layers,batch_size,self.num_hidden))
            
            return hidden
        except Exception as e:
            logger.error(f"Error Initializing and returning the initial hidden state: {str(e)}")
        

## Instance of the Model

In [ ]:
model = CharModel(
    all_chars=all_characters,
    num_hidden=512,
    num_layers=3,
    drop_prob=0.5,
    use_gpu=True
)

### Optimizer and Loss

In [ ]:
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
criterion = nn.CrossEntropyLoss()

## Training Data and Validation Data

In [ ]:
# percentage of data to be used for training
train_percent = 0.5

In [ ]:
int(len(encoded_text) * (train_percent))

In [ ]:
train_ind = int(len(encoded_text) * (train_percent))

In [ ]:
train_data = encoded_text[:train_ind]
val_data = encoded_text[train_ind:]

# Training the Network

## Variables

In [ ]:
# Epochs to train for
epochs = 30
# batch size 
batch_size = 128
# Length of sequence
seq_len = 100
# for printing report purposes
# always start at 0
tracker = 0
# number of characters in text
num_char = max(encoded_text)+1

In [ ]:
mlflow.set_experiment(EXPERIMENT_SET)

In [ ]:
mlflow.start_run(run_name = RUN_NAME)

mlflow.log_param("epochs", epochs)
mlflow.log_param("batch_size", batch_size)

# Set model to train
model.train()

# Check to see if using GPU
if model.use_gpu:
    torch.cuda.manual_seed_all(0)
    model.cuda()

for i in range(epochs):
    
    hidden = model.hidden_state(batch_size)
    
    
    for x,y in generate_batches(train_data, batch_size, seq_len):
        
        tracker += 1
        
        # One Hot Encode incoming data
        x = one_hot_encoder(x, num_char)
        
        # Convert Numpy Arrays to Tensor
        inputs = torch.from_numpy(x)
        targets = torch.from_numpy(y)
        
        # Adjust for GPU if necessary
        if model.use_gpu:
            inputs = inputs.cuda()
            targets = targets.cuda()
            
        # Reset Hidden State
        hidden = tuple([state.data for state in hidden])
        
        model.zero_grad()
        
        lstm_output, hidden = model.forward(inputs, hidden)
        loss = criterion(lstm_output, targets.view(batch_size*seq_len).long())
        
        loss.backward()
        
        # Clipping gradients to avoid explosion
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=5)
        
        optimizer.step()
        
        if tracker % 100 == 0:
            val_hidden = model.hidden_state(batch_size)
            val_losses = []
            model.eval()
            
            for x,y in generate_batches(val_data, batch_size, seq_len):
                x = one_hot_encoder(x, num_char)
                inputs = torch.from_numpy(x)
                targets = torch.from_numpy(y)
                
                if model.use_gpu:
                    inputs = inputs.cuda()
                    targets = targets.cuda()
                
                val_hidden = tuple([state.data for state in val_hidden])
                
                lstm_output, val_hidden = model.forward(inputs, val_hidden)
                val_loss = criterion(lstm_output, targets.view(batch_size*seq_len).long())
        
                val_losses.append(val_loss.item())
            
  
            mlflow.log_metric("Val Loss", val_loss.item(), step=tracker)
        
            model.train()
            
    logger.info(f"Epoch: {i} Step: {tracker} Val Loss: {val_loss.item()}")


mlflow.end_run()

## Saving the Model

In [ ]:
model_name = MODEL_NAME

In [ ]:
torch.save(model.state_dict(), f'models/{model_name}')
logger.info("Model saved")

## Load Model

In [ ]:
model = CharModel(
    all_chars=all_characters,
    num_hidden=512,
    num_layers=3,
    drop_prob=0.5,
    use_gpu=False
)

In [ ]:
model.load_state_dict(torch.load(f'models/{model_name}'))
model.eval()

# Generating Predictions

--------

In [ ]:
def predict_next_char(model, char, hidden=None, k=1):
    """
    Predicts the next character given an input character and the current hidden state.

    This method encodes the input character, feeds it through the trained character-level 
    language model (e.g., LSTM), and samples from the top-k most probable characters 
    to determine the next one. It also returns the updated hidden state for sequential prediction.

    Args:
        char: The input character to start prediction from.
        hidden: Current hidden state of the model. Each tensor has shape (num_layers, batch_size, num_hidden).
            If None, a new hidden state should be initialized before calling this method.
        k: Number of top predictions to sample from.

    Returns:
        A tuple containing the predicted next character and the updated hidden state.
    """
    try:
        encoded_text = model.encoder[char]
        encoded_text = np.array([[encoded_text]])
        encoded_text = one_hot_encoder(encoded_text, len(model.all_chars))
        inputs = torch.from_numpy(encoded_text)
    
        if(model.use_gpu):
            inputs = inputs.cuda()  

        hidden = tuple([state.data for state in hidden])
        lstm_out, hidden = model(inputs, hidden)        
        probs = F.softmax(lstm_out, dim=1).data
    
        if(model.use_gpu):
            probs = probs.cpu()

    # Getting the top 'k' for next char probs
        probs, index_positions = probs.topk(k)        
        index_positions = index_positions.numpy().squeeze()
        probs = probs.numpy().flatten()
        probs = probs/probs.sum()
        char = np.random.choice(index_positions, p=probs)    
    
        return model.decoder[char], hidden
    except Exception as e:
            logger.error(f"Error predicting next char: {str(e)}")

In [ ]:
def generate_text(model, size, seed='The', k=1):
    """
    Generates a sequence of text using the trained character-level language model.

    Starting from a seed string, this method uses the model to predict the next character
    one at a time, feeding each predicted character back into the model. It continues
    this process until the desired output length is reached.

    Args:
        seed: The initial sequence of characters used to start the text generation.
        size: The number of characters to generate after the seed.
        k: Number of top character predictions to consider for sampling at each step.

    Returns:
        The full generated text including the seed and the newly predicted characters.
    """
    try:
        if(model.use_gpu):
            model.cuda()
        else:
            model.cpu()
            
        model.eval()
        output_chars = [c for c in seed]
        hidden = model.hidden_state(1)
        
        for char in seed:
            char, hidden = predict_next_char(model, char, hidden, k=k)

        output_chars.append(char)
        for i in range(size):
            char, hidden = predict_next_char(model, output_chars[-1], hidden, k=k)
            output_chars.append(char)
            
        return ''.join(output_chars)
    except Exception as e:
            logger.error(f"Error making predictions: {str(e)}")


#### Generating a text with 1000 chars starting with word 'Confidence'

In [ ]:
print(generate_text(model, 1000, seed='Confidence ', k=3))

#### Generating a text with 1000 chars starting with word 'Love'

In [ ]:
print(generate_text(model, 1000, seed='Love ', k=3))

Built with ❤️ using Z by HP AI Studio.